In [5]:
## 1. 환경 설정 및 데이터 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 시각화 그래프가 노트북 안에 나타나도록 설정
%matplotlib inline
# 그래프 스타일 설정
sns.set(style="whitegrid")

# 데이터 경로 설정 (현재 노트북 위치의 상위 폴더(../)에 있는 data 폴더 접근)
train_path = '../data/titanic/train.csv'
test_path = '../data/titanic/test.csv'

# 파일 존재 여부 확인 후 불러오기
if os.path.exists(train_path):
    print(f"파일을 찾았습니다: {train_path}")
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    print("데이터 로드 성공!")
    
    # 데이터의 처음 5줄 확인
    display(train_df.head())
else:
    print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {train_path}")
    print(f"현재 작업 경로: {os.getcwd()}")

파일을 찾았습니다: ../data/titanic/train.csv
데이터 로드 성공!


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
## 1단계: 고급 피처 엔지니어링 (데이터 다시 불러오기부터 시작)
import pandas as pd
import numpy as np

# 데이터 다시 로드
train_df = pd.read_csv('../data/titanic/train.csv')
test_df = pd.read_csv('../data/titanic/test.csv')
combine = [train_df, test_df]

# 1. 호칭(Title) 추출 및 매핑
for dataset in combine:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace(['Mlle', 'Ms'], 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

# 2. 성별(Sex) 매핑
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

# 3. 나이(Age) 정교하게 채우기 (호칭별 중간값 사용)
# (비어있는 나이를 "Mr"는 Mr의 평균 나이로, "Miss"는 Miss의 평균 나이로 채움)
for dataset in combine:
    # 0:Male, 1:Female / 1~5: Title
    for i in range(0, 2):
        for j in range(1, 6):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Title'] == j)]['Age'].dropna()
            age_guess = guess_df.median()
            # 반올림
            age_guess = int( age_guess/0.5 + 0.5 ) * 0.5
            
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Title == j),\
                    'Age'] = age_guess
    dataset['Age'] = dataset['Age'].astype(int)

# 4. 나이 구간(AgeBand) 나누기 (5개 구간)
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
# 구간을 숫자로 변경
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

# 5. 가족 규모(FamilySize) & 혼자 여부(IsAlone)
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

# 6. 운임(Fare) 채우기 & 구간(FareBand) 나누기
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)

for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

# 7. 승선항(Embarked) 채우기
freq_port = train_df.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

# 8. 불필요한 피처 삭제 (AgeBand, FareBand는 확인용이었으므로 삭제)
train_df = train_df.drop(['AgeBand', 'FareBand', 'Name', 'Ticket', 'Cabin', 'PassengerId', 'SibSp', 'Parch'], axis=1)
test_df = test_df.drop(['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1)

print("고급 전처리 완료!")
display(train_df.head())

ValueError: cannot convert float NaN to integer

In [ ]:
## 2단계: 앙상블 모델링 (Voting)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC

# 데이터 준비
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()

# 1. 개별 모델 정의 (최적 파라미터 적용)
# (이미 튜닝된 파라미터 값을 적용했습니다)
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=6, min_samples_leaf=1, random_state=42)
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, max_depth=3, random_state=42)
svc_clf = SVC(probability=True, random_state=42) # 확률 기반 투표를 위해 probability=True

# 2. 보팅 분류기(Voting Classifier) 정의 (Soft Voting)
# 3개의 모델이 머리를 맞대고 상의해서 결과를 냅니다.
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('gb', gb_clf), ('svc', svc_clf)],
    voting='soft'
)

# 3. 학습 및 예측
voting_clf.fit(X_train, Y_train)
final_predictions = voting_clf.predict(X_test)

print("앙상블 모델 예측 완료!")

# 4. 제출 파일 저장
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": final_predictions
    })
submission.to_csv('submission_ensemble.csv', index=False)
print("submission_ensemble.csv 저장 완료! 이 파일을 제출해보세요.")